# Decision Tree on the famous Iris flower data set
Here is my own implementation of this classic machine learning algorithm. I tried to use as few libraries as possible.

In [1]:
import csv
from random import shuffle

# classical iris flower data set
with open('IRIS.csv') as csvfile:
     data_set = list(csv.reader(csvfile))
        
shuffle(data_set)
cut = int(0.8 * len(data_set))
training_data = data_set[:cut]
testing_data = data_set[cut:]

In [2]:
# no need to use the "actual" labels
header = ["a", "b", "c", "d", "e", "f", "g", "h"]

In [3]:
# the probability of choosing the wrong answer randomly
def gini_impurity(data):
    classes = {}
    for instance in data:
        if instance[-1] not in classes:
            classes.update({instance[-1] : 1})
        else:
            classes.update({instance[-1] : (classes.get(instance[-1]) + 1)})
    
    impurity = 0
    if len(classes) == 1:
        return 0
    for i in classes:
        prob = len(classes) / classes.get(i)
        impurity += prob * (1 - prob)
    
    return impurity

In [4]:
print(gini_impurity(training_data))

0.3000286974048879


In [5]:
# find out how much the "uncertainty" can be reduced by a split
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini_impurity(left) - (1 - p) * gini_impurity(right)

In [6]:
class Question:
    
    def __init__(self, index, attribute):
        self.index = index
        self.attribute = attribute   
    
    # all features are numerical
    def decide(self, instance):
            return (instance[self.index] > self.attribute)
    
    # split data set into a wrong list (left) and a right list (right)
    def separate(self, data):
        right = []
        left = []
        for instance in data:
            if self.decide(instance):
                right.append(instance)
            else:
                left.append(instance)
        
        return left, right

In [7]:
class Decision_Tree:
    def __init__(self, question, left, right):
        self.question = question
        self.left = left
        self.right = right
    
    def predict(self, instance):
        if self.question.decide(instance):
            return self.right.predict(instance)
        else:
            return self.left.predict(instance)

In [8]:
class Leaf(Decision_Tree):
    def __init__(self, label):
        self.label = label
    def predict(self, instance):
        # the guess has to be made here
        return self.label

In [9]:
def create_decision_tree(data):
    if gini_impurity(data) == 0:
        return Leaf(data[0][-1])
    questions = []
    # collect all possible questions
    for instance in data:
        for feature in header[:-1]:
            questions.append(Question(header.index(feature), instance[header.index(feature)]))
    best_information_gain = 0
    current_impurity = gini_impurity(data)
    best_question = None
    # finding out which question does best on increasing the information gain
    for question in questions:
        left, right = question.separate(data)
        gain = info_gain(left, right, current_impurity)
        if gain > best_information_gain:
            best_information_gain = gain
            best_question = question
    
    # if no question can increase the information gain
    if best_question == None:
        return Leaf(data[0][-1])
    left, right = best_question.separate(data)
    
    return Decision_Tree(best_question, create_decision_tree(left), create_decision_tree(right))    

In [10]:
# constructing the decision tree by use of the training data from the data set
decision_tree = create_decision_tree(training_data)
falses = 0
for instance in testing_data:
    print("prediction: " + decision_tree.predict(instance[:-1]) + ", actual: " + instance[-1])
    if decision_tree.predict(instance[:-1]) != instance[-1]:
        falses += 1

error_rate = falses / len(testing_data)
print("\nThe error rate is:", error_rate)

prediction: versicolor, actual: versicolor
prediction: versicolor, actual: versicolor
prediction: versicolor, actual: versicolor
prediction: setosa, actual: setosa
prediction: setosa, actual: versicolor
prediction: setosa, actual: setosa
prediction: setosa, actual: virginica
prediction: setosa, actual: setosa
prediction: virginica, actual: virginica
prediction: versicolor, actual: versicolor
prediction: virginica, actual: virginica
prediction: versicolor, actual: versicolor
prediction: setosa, actual: setosa
prediction: versicolor, actual: virginica
prediction: versicolor, actual: versicolor
prediction: setosa, actual: virginica
prediction: virginica, actual: virginica
prediction: versicolor, actual: virginica
prediction: virginica, actual: virginica
prediction: versicolor, actual: virginica

The error rate is: 0.3
